In [2]:
import torch
print(torch.__version__)

2.8.0+cu126


In [3]:
from torch import nn

In [9]:
# 1. Define the model

class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__() # This line is required for all Python classes!
    
        # Define the model here
        self.middle_layer = nn.Linear(3, 4)
        self.final_layer = nn.Linear(4, 1)
        
    def forward(self, x):
        # return the prediction here
        intermediate = self.middle_layer.forward(x)
        return self.final_layer.forward(intermediate)

In [16]:
model = SimpleModel()
model.train()

SimpleModel(
  (middle_layer): Linear(in_features=3, out_features=4, bias=True)
  (final_layer): Linear(in_features=4, out_features=1, bias=True)
)

In [17]:
# Das ist eine einfache Lineare Lösung

data_point = torch.tensor([45, 1.5, 1.7]) # Dimension very important Eingabe (3,)
true_answer  = torch.tensor(0.75)                 # Ziel als Skalar

prediction = model(data_point)  # Forward function is calling

print(prediction.item())

-6.4816975593566895


In [19]:
# optimizer und hyperparameter
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
num_it = 1000

In [22]:
# 2 Training

for i in range(num_it):
    optimizer.zero_grad()                     # 1) Gradienten zurücksetzen
    pred = model(data_point)                  # 2) Vorwärts (Predictions)
    error = nn.functional.mse_loss(pred, true_answer) # 3) Error (Loss)
    error.backward() # we use the error und calculte the derivative
    
    # use the derivative to update the parameter
    optimizer.step() # Update
    
    if (i+1) % 100 == 0:
        print(f"{i+1:4d}: loss={error.item():.6f}, pred={pred.item():.4f}")

/tmp/ipykernel_171642/4073018911.py:6: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  error = nn.functional.mse_loss(pred, true_answer) # 3) Error (Loss)


 100: loss=0.000000, pred=0.7500
 200: loss=0.000000, pred=0.7500
 300: loss=0.000000, pred=0.7500
 400: loss=0.000000, pred=0.7500
 500: loss=0.000000, pred=0.7500
 600: loss=0.000000, pred=0.7500
 700: loss=0.000000, pred=0.7500
 800: loss=0.000000, pred=0.7500
 900: loss=0.000000, pred=0.7500
1000: loss=0.000000, pred=0.7500


## copy

In [23]:
import torch
import torch.nn as nn

# --- Model wie vorher ---
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.middle_layer = nn.Linear(3, 4)
        self.final_layer  = nn.Linear(4, 1)
    def forward(self, x):
        return self.final_layer(self.middle_layer(x))

model = SimpleModel()
model.train()

# --- Daten ---
x = data_point.unsqueeze(0)      # (1,3)
y = true_answer.unsqueeze(0).unsqueeze(1)  # (1,1)

# --- Optimizer & Hyperparameter ---
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
num_it = 1000

for i in range(num_it):
    optimizer.zero_grad()
    pred = model(x)              # (1,1)
    loss = nn.functional.mse_loss(pred, y)
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
        print(f"{i+1:4d}: loss={loss.item():.6f}, pred={pred.item():.4f}")


 100: loss=0.000000, pred=0.7500
 200: loss=0.000000, pred=0.7500
 300: loss=0.000000, pred=0.7500
 400: loss=0.000000, pred=0.7500
 500: loss=0.000000, pred=0.7500
 600: loss=0.000000, pred=0.7500
 700: loss=0.000000, pred=0.7500
 800: loss=0.000000, pred=0.7500
 900: loss=0.000000, pred=0.7500
1000: loss=0.000000, pred=0.7500


## Model testen

In [24]:
model.eval()

SimpleModel(
  (middle_layer): Linear(in_features=3, out_features=4, bias=True)
  (final_layer): Linear(in_features=4, out_features=1, bias=True)
)

In [25]:
with torch.no_grad():
    test_point = torch.tensor([50.0, 2.0, 1.0]).unsqueeze(0)  # (1,3)
    prediction = model(test_point)
    print("Vorhersage:", prediction.item())

Vorhersage: 0.5990891456604004


### Loss auf Testdaten messen

In [28]:
true_answer_test  = torch.tensor(0.75)  

with torch.no_grad():
    pred_test = model(test_point)
    loss_test = nn.functional.mse_loss(pred_test, true_answer_test.unsqueeze(0).unsqueeze(1))
    print("Test Loss:", loss_test.item())

Test Loss: 0.02277408540248871


### Save the model

In [29]:
# In PyTorch speichert man normalerweise nur die Gewichte (state_dict), nicht das ganze Modellobjekt.
torch.save(model.state_dict(), "simple_model.pth")

In [ ]:
# Komplettes Modell speichern (seltener empfohlen)
torch.save(model, "simple_model_full.pth")

## Model hochladen

In [ ]:
# Neues Modell mit gleicher Architektur erstellen
model2 = SimpleModel()

# Gewichte laden
model2.load_state_dict(torch.load("simple_model.pth"))

# In den Evaluationsmodus setzen
model2.eval()


In [ ]:
model3 = torch.load("simple_model_full.pth")
model3.eval()

* torch.save(model.state_dict()) → saubere Variante (empfohlen).

* torch.save(model) → nur für schnelle Tests, aber weniger portabel.